In [3]:
!pip install webdriver-manager
!pip install pandas-datareader
!pip install selenium

  Using cached python_dotenv-0.21.0-py3-none-any.whl (18 kB)


  Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)
     ---------------------------------------- 6.3/6.3 MB 26.9 MB/s eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 23.4 MB/s eta 0:00:00
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
from datetime import datetime
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
servico = Service(ChromeDriverManager().install())
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas.tseries.offsets as ts
from time import sleep


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.58M/6.58M [00:00<00:00, 24.3MB/s]


In [6]:
navegador = webdriver.Chrome(service=servico)
navegador.get("https://www.anbima.com.br/informacoes/curvas-debentures/")
sleep(1)
bot_tela = navegador.find_element(By.XPATH,'//*[@id="cinza50"]/form/div/fieldset/font/input[1]').click()
sleep(1)
bot_consulta = navegador.find_element(By.XPATH,'//*[@id="cinza50"]/form/div/table/tbody/tr/td/img').click()
sleep(3)
div = navegador.find_element(By.ID,'Curvas')

tab_tratada = div.get_attribute('innerHTML').replace(
    '\n','').replace('\t','').replace(
    '                    ','').replace(
    '                ','').replace(
    '    ','')#.replace('/td','').replace('/tr','')

# Obtendo o conteúdo da página em formato de texto
soup = BeautifulSoup(tab_tratada,"html.parser")

Curva_Anbima = pd.DataFrame(columns=['Vértices (Anos)', 'AAA', 'AA', 'A'])

count = 1

for row in soup.find_all('tr'):
    columns = row.find_all('td')  # em html uma coluna da tabela é representada pela tag <td>
#     print(count, len(columns))
    count += 1
    if(columns != []):
        Vertices= columns[0].text.strip(' ')
        AAA = columns[1].text.strip(' ')
        AA = columns[2].text.strip(' ')
        A = columns[3].text.strip(' ')
        Curva_Anbima = pd.concat([Curva_Anbima, pd.DataFrame.from_records([{'Vértices (Anos)': Vertices
                                                        ,  'AAA': AAA, 'AA': AA, 'A': A
                                                        }])],ignore_index=True)
    
navegador.quit()  


valores_anbima = Curva_Anbima.iloc[:20]
valores_anbima['AAA'] =valores_anbima['AAA'].str.replace(',','.').astype('float')
valores_anbima['AA'] =valores_anbima['AA'].str.replace(',','.').astype('float')
valores_anbima['A'] =valores_anbima['A'].str.replace(',','.').astype('float')
valores_anbima['Vértices (Anos)'] =valores_anbima['Vértices (Anos)'].str.replace(',','.').astype('float')
   
valores_anbima


C:\Users\BR0443447818\AppData\Local\Temp\ipykernel_12684\1318162109.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valores_anbima['AAA'] =valores_anbima['AAA'].str.replace(',','.').astype('float')
C:\Users\BR0443447818\AppData\Local\Temp\ipykernel_12684\1318162109.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valores_anbima['AA'] =valores_anbima['AA'].str.replace(',','.').astype('float')
C:\Users\BR0443447818\AppData\Local\Temp\ipykernel_12684\1318162109.py:48: SettingWithCopyWarning: 
A value 

,index,Vértices (Anos),AAA,AA,A
0,0,0.5,0.7525,1.1964,1.2964
1,1,1.0,0.8861,1.3300,1.4300
2,2,1.5,1.0073,1.4511,1.5511
3,3,2.0,1.1173,1.5612,1.6612
4,4,2.5,1.2174,1.6613,1.7613
5,5,3.0,1.3086,1.7524,1.8524
6,6,3.5,1.3917,1.8356,1.9356
7,7,4.0,1.4676,1.9115,2.0115
8,8,4.5,1.5371,1.9810,2.0810
9,9,5.0,1.6007,2.0446,2.1446
